# Stock Price Prediction with LSTM - Full Pipeline

This notebook demonstrates the complete pipeline for stock price prediction using LSTM neural networks.


## 1. Setup and Imports


In [ ]:
import sys
import os

# Add parent directory to path
sys.path.insert(0, os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.data_downloader import StockDataDownloader
from src.feature_engineering import FeatureEngineer
from src.model_training import ModelTrainer
from src.strategy import TradingStrategy
from src.backtester import Backtester
from src.visualization import Visualizer


## 2. Download Stock Data


In [ ]:
# Download 3 years of AAPL data
ticker = "AAPL"
downloader = StockDataDownloader(ticker, period="3y", interval="1d")
data = downloader.download()

# Display first few rows
print(f"Downloaded {len(data)} days of data")
print(f"Date range: {data.index[0]} to {data.index[-1]}")
data.head()


## 3. Feature Engineering


In [ ]:
# Create technical indicators
engineer = FeatureEngineer(lookback_window=60)
features = engineer.create_features(data)
target = engineer.create_target(data['Close'])

print(f"Created {len(features.columns)} features")
print(f"\nFeature columns:")
for col in features.columns:
    print(f"  - {col}")

features.head()


## 4. Prepare Sequences for LSTM


In [ ]:
# Create sequences
X_train, X_test, y_train, y_test = engineer.create_sequences(features, target, test_size=0.2)

print(f"Training samples: {X_train.shape}")
print(f"Test samples: {X_test.shape}")


## 5. Train LSTM Model


In [ ]:
# Initialize trainer
trainer = ModelTrainer(lookback_window=60)

# Train model
history = trainer.train(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    verbose=1
)


## 6. Evaluate Model


In [ ]:
# Evaluate on test set
metrics = trainer.evaluate(X_test, y_test)

print("\nModel Performance:")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1 Score: {metrics['f1_score']:.4f}")


## 7. Generate Predictions


In [ ]:
# Generate predictions with confidence
predictions, confidence = trainer.predict_with_confidence(X_test)

print(f"Generated {len(predictions)} predictions")
print(f"Average confidence: {confidence.mean():.4f}")
print(f"\nPrediction distribution:")
print(f"  Up predictions: {sum(predictions == 1)}")
print(f"  Down predictions: {sum(predictions == 0)}")


## 8. Generate Trading Signals


In [ ]:
# Create trading strategy
# Note: Uses lenient BUY threshold (min 0.4) with fallback to 0.3 if no signals
strategy = TradingStrategy(
    buy_threshold=0.5,  # Lowered to generate more BUY signals
    sell_threshold=0.5,  # Balanced threshold for SELL signals
    commission=0.001,
    initial_capital=100000
)

# Generate signals
signals = strategy.generate_signals(predictions, confidence)

print(f"\nTrading Signals:")
print(f"  BUY signals: {sum(signals == 1)}")
print(f"  SELL signals: {sum(signals == -1)}")
print(f"  HOLD signals: {sum(signals == 0)}")


## 9. Run Backtest


In [ ]:
# Get prices for test period
test_start_idx = len(data) - len(X_test)
test_prices = data['Close'].iloc[test_start_idx:]
test_dates = data.index[test_start_idx:]

# Run backtest
backtester = Backtester(strategy)
results = backtester.backtest(test_prices, signals, dates=test_dates)

# Display results
results.head()


## 10. Calculate Performance Metrics


In [ ]:
# Calculate metrics
performance_metrics = backtester.calculate_metrics()

print("\nBacktest Performance Metrics:")
for key, value in performance_metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")


## 11. Visualizations


In [ ]:
# Create visualizer
visualizer = Visualizer(save_dir="../results/plots")

# Plot price data
visualizer.plot_price_data(data, ticker, save=True)

# Plot technical indicators
visualizer.plot_technical_indicators(data, features, ticker, save=True)

# Plot training history
visualizer.plot_training_history(history, save=True)

# Plot equity curve
visualizer.plot_equity_curve(results, ticker, save=True)

# Plot signals
signal_data = data.iloc[test_start_idx:]
visualizer.plot_signals(signal_data, signals, ticker, save=True)

# Plot performance metrics
visualizer.plot_performance_metrics(performance_metrics, ticker, save=True)

# Plot confusion matrix
visualizer.plot_confusion_matrix(y_test, predictions, save=True)


## 12. Save Results


In [ ]:
# Save model
model_path = f"../models/saved_models/{ticker}_model"
trainer.save_model(model_path)

# Save backtest results
results.to_csv(f"../results/{ticker}_backtest_results.csv", index=False)

# Save performance metrics
metrics_df = pd.DataFrame([performance_metrics])
metrics_df.to_csv(f"../results/{ticker}_performance_metrics.csv", index=False)

# Save trades
trades = backtester.get_trades()
if len(trades) > 0:
    trades.to_csv(f"../results/{ticker}_trades.csv", index=False)

print("\nAll results saved successfully!")
